#  Feed forward neural network

In [4]:
import tensorflow as tf
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

In [6]:
# %% Load and preprocess the dataset
iris = datasets.load_iris()
X = iris.data  # Features

# One-hot encode the target labels
y = to_categorical(iris.target)
print("Shape of y after one-hot encoding:", y.shape)

Shape of y after one-hot encoding: (150, 3)


In [7]:
# Split the data into training and test sets (80:20 ratio)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training set into training and validation sets (90:10 ratio of the training set)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [8]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [14]:
# %% Build the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax') 
])

# Model summary
model.summary()

c:\Users\sbhar\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 739 (2.89 KB)

 Trainable params: 739 (2.89 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile the model with SGD optimizer and cross-entropy loss
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=8,
                    validation_data=(X_val, y_val), verbose=1)


Epoch 1/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.1977 - loss: 1.1853 - val_accuracy: 0.1667 - val_loss: 1.1768
Epoch 2/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3231 - loss: 1.1001 - val_accuracy: 0.4167 - val_loss: 1.0927
Epoch 3/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5803 - loss: 0.9977 - val_accuracy: 0.5833 - val_loss: 1.0219
Epoch 4/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6185 - loss: 0.9458 - val_accuracy: 0.5000 - val_loss: 0.9605
Epoch 5/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5717 - loss: 0.9233 - val_accuracy: 0.4167 - val_loss: 0.9045


In [16]:
# Evaluate the model on the validation and test sets
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

In [17]:
# Print the results
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print(f"Testing Accuracy: {test_accuracy * 100:.2f}%")


Validation Accuracy: 41.67%
Testing Accuracy: 66.67%


# CNN

In [ ]:
from glob import glob
from sklearn.model_selection import train_test_split
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

In [ ]:
import tensorflow as tf

# Load dataset with a 60:40 split, where 40% will be further split into validation and test sets
train_ds = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/rice-image-dataset/Rice_Image_Dataset',
    validation_split=0.4,
    subset='training',
    seed=123,
    image_size=(125, 125),
    batch_size=8
)

test_val_ds = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/rice-image-dataset/Rice_Image_Dataset',
    validation_split=0.4,
    subset='validation',
    seed=123,
    image_size=(125, 125),
    batch_size=8
)

# Further split test_val_ds into validation and test sets (50% each)
val_size = int(0.5 * len(test_val_ds))
val_ds = test_val_ds.take(val_size)
test_ds = test_val_ds.skip(val_size)

# Proceed with model training and evaluation using train_ds, val_ds, and test_ds


In [ ]:
# Gather all image paths
dirs = glob('/kaggle/input/rice-image-dataset/Rice_Image_Dataset/*')
paths = []
for directory in dirs:
    paths.extend(glob('{}/*'.format(directory)))
print(f"Total images: {len(paths)}")

In [ ]:
# Split paths into train, validation, and test sets
train_paths, test_paths = train_test_split(paths, test_size=0.2, random_state=42)
train_paths, val_paths = train_test_split(train_paths, test_size=0.1, random_state=42)

In [ ]:
# Define ImageDataGenerator for each split
train_datagen = ImageDataGenerator(rescale=1.0/255)
val_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
# Define generators for the splits
def path_to_generator(paths, datagen, batch_size=32, target_size=(150, 150)):
    def preprocess_image(path):
        img = tf.keras.preprocessing.image.load_img(path, target_size=target_size)
        img = tf.keras.preprocessing.image.img_to_array(img)
        return img / 255.0
    
    dataset = tf.data.Dataset.from_tensor_slices(paths)
    dataset = dataset.map(lambda path: preprocess_image(path))
    return dataset.batch(batch_size)

In [ ]:
train_generator = path_to_generator(train_paths, train_datagen)
val_generator = path_to_generator(val_paths, val_datagen)
test_generator = path_to_generator(test_paths, test_datagen)

In [ ]:
# Model setup and training
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(5, activation='softmax')
])


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Training
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator
)

In [ ]:
# Evaluation
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


# LSTM and FFNN

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


In [ ]:
# Load data from CSV
infosys_data = pd.read_csv("infosys_data.csv")
sbi_data = pd.read_csv("sbi_data.csv")
ioc_data = pd.read_csv("ioc_data.csv")

# Only keep the 'Close' column for training
data = infosys_data[['Close']]


In [ ]:
# Normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

# Prepare sequences
sequence_length = 60  # Use 60 previous days to predict the next day
X, y = [], []
for i in range(sequence_length, len(data_scaled)):
    X.append(data_scaled[i-sequence_length:i, 0])
    y.append(data_scaled[i, 0])

X, y = np.array(X), np.array(y)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))  # LSTM expects 3D input


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
# Define the LSTM model
lstm_model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    LSTM(50),
    Dense(1)
])

lstm_model.compile(optimizer='adam', loss='mean_squared_error')
lstm_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/50


c:\Users\sbhar\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - loss: 0.2934 - val_loss: 0.2236
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.2278 - val_loss: 0.2441
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2398 - val_loss: 0.2269
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.2250 - val_loss: 0.2247
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.2258 - val_loss: 0.2275
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.2273 - val_loss: 0.2244
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.2241 - val_loss: 0.2234
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.2235 - val_loss: 0.2246
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2243 - val_loss: 0.2237
Epoch 10/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.2233 - val_loss: 0.2230
Epoch 11/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2230 - val_loss: 0.2232
Epoch 12/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.2234 - val_loss: 0.2230
Epoch 13/50


In [36]:
lstm_model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 4, 50)          │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 91,955 (359.20 KB)

 Trainable params: 30,651 (119.73 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 61,304 (239.47 KB)

In [37]:
# Reshape data for FFNN
X_ffnn = np.reshape(X, (X.shape[0], X.shape[1]))

# Split FFNN data
X_ffnn_train, X_ffnn_test, y_ffnn_train, y_ffnn_test = train_test_split(X_ffnn, y, test_size=0.2, random_state=42)

# Define FFNN model
ffnn_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_ffnn_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1)
])

ffnn_model.summary()

ffnn_model.compile(optimizer='adam', loss='mean_squared_error')
ffnn_model.fit(X_ffnn_train, y_ffnn_train, epochs=5, batch_size=32, validation_data=(X_ffnn_test, y_ffnn_test))


c:\Users\sbhar\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_22 (Dense)                │ (None, 128)            │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,961 (35.00 KB)

 Trainable params: 8,961 (35.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.2513 - val_loss: 0.2590
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2429 - val_loss: 0.2440
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2379 - val_loss: 0.2316
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.2327 - val_loss: 0.2256
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.2279 - val_loss: 0.2255


In [ ]:
# LSTM Predictions
lstm_predictions = lstm_model.predict(X_test)
lstm_predictions = scaler.inverse_transform(lstm_predictions)

# FFNN Predictions
ffnn_predictions = ffnn_model.predict(X_ffnn_test)
ffnn_predictions = scaler.inverse_transform(ffnn_predictions.reshape(-1, 1))

# Rescale y_test for comparison
y_test_scaled = scaler.inverse_transform(y_test.reshape(-1, 1))


In [ ]:
from sklearn.metrics import mean_squared_error

lstm_mse = mean_squared_error(y_test_scaled, lstm_predictions)
ffnn_mse = mean_squared_error(y_test_scaled, ffnn_predictions)

print(f"LSTM MSE: {lstm_mse}")
print(f"FFNN MSE: {ffnn_mse}")
